<a href="https://colab.research.google.com/github/emasri/tcol/blob/master/td3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Chcek notebook's internet speed

In [ ]:
#@markdown

!sudo apt install speedtest-cli
!speedtest-cli

###Time out preventer

Run this code to prevent Notebook Timeout

In [ ]:
#@markdown

#from os import makedirs
#makedirs("/root/.config/rclone", exist_ok = True)
import IPython
from google.colab import output

display(IPython.display.Javascript('''
function ClickConnect(){
  btn = document.querySelector("colab-connect-button")
  if (btn != null){
    console.log("Click colab-connect-button"); 
    btn.click() 
    }
    
    btn = document.getElementById('ok')
    if (btn != null){
    console.log("Click reconnect"); 
    btn.click() 
    }
  }
  
setInterval(ClickConnect,60000)
''')) 

### Start session

In [ ]:
#@markdown


!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent

!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)

sett = {'allow_multiple_connections_per_ip': True,
  'active_downloads': -1}

ses.apply_settings(sett)

downloads = []

###Mount drive

In [ ]:
#@markdown

from google.colab import drive
drive.mount('/content/drive')

###Add torrent file

In [ ]:
#@markdown

from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Library/Downloads",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

###Add multi torrent magnet

In [ ]:
#@markdown 
import time
savepath = "/content/drive/My Drive/Library/Downloads"
params = {"save_path": savepath}
magnet_link = ""
handle=""

while True:
    magnet_link = input("Enter Magnet Link Or Type e: ")
    if magnet_link.lower() == "e":
        break
    handle = lt.add_magnet_uri(ses, magnet_link, params)
    handle.stop_when_ready(True)
    downloads.append(
        handle
    )

print("Save path: "+savepath)

###Download metadata multi torrent


In [ ]:
#@markdown 
checks=[]
for h in downloads:
  h.resume()
  checks.append(h)

while (checks):
  for h in checks:
    if h.has_metadata():
      checks.remove(h)

ses.pause()

###List all files multi 

In [ ]:
#@markdown 
print("All files:")

for h in downloads:
  print("")
  print("  "+h.name())
  if(h.has_metadata()):
    torinfo = h.get_torrent_info()
    torfiles = torinfo.files()
    for x in range(torfiles.num_files()):
      print("   {:3d}".format(x),torfiles.file_name(x))
  else:
    print("    Metadata not recieved yet.")


###Filter multi file

In [ ]:
#@markdown 

#@markdown 

skip_list = ['RARBG.txt',
             'RARBG_DO_NOT_MIRROR.exe',
             'readme.txt'
             ]

for h in downloads:
  if(h.has_metadata()):
    torinfo = h.get_torrent_info()
    files = torinfo.files()
    for x in range(files.num_files()):
      if any(i in files.file_name(x) for i in skip_list):
        h.file_priority(x,0)


###Check download files multi 

In [ ]:
#@markdown
print("Download files:")

for h in downloads:
  print("")
  print("  "+h.name())
  if(h.has_metadata()):
    priorities = h.file_priorities()
    torinfo = h.get_torrent_info()
    files = torinfo.files()
    for x in range(files.num_files()):
      if (priorities[x]>0):
        print("   Download", files.file_name(x))
  else:
    print("    Metadata not recieved yet.")


###Add single torrent magnet

In [ ]:
import time

ses.resume()

savepath = "/content/drive/My Drive/Library/Downloads"

#@markdown

#@markdown

#@markdown

#@markdown

#@markdown Choose part
Part = 0 #@param {type:"slider", min:0, max:10, step:1}
if Part > 0:
  savepath+= ("/p"+str(Part))

params = {"save_path": savepath}

#@markdown

#@markdown Enter magnet link
MagnetLink = "" #@param {type:"string"}

handle = lt.add_magnet_uri(ses, MagnetLink, params)
downloads.append(
    handle
)

print("Save path: "+savepath)

while (not handle.has_metadata()):
  time.sleep(0.2)

ses.pause () 

###Sequential download

In [ ]:
from traitlets.traitlets import SequenceTypes
#@markdown 

SequentialDownload = False #@param{type:"boolean"}

handle.set_sequential_download(SequentialDownload)  

Check sequential download

In [ ]:
#@markdown 

#@markdown 

status = handle.status()
print(status.sequential_download)

###File list single

In [ ]:
#@markdown 

#@markdown 

torinfo = handle.get_torrent_info()
files = torinfo.files()

for x in range(files.num_files()):
  print(x,files.file_name(x))

###Choose seasons

In [ ]:
s_list=["S01","S02","S03","S04","S05","S06","S07","S08","S09","S10","S11","S12"]
check_list =["False","False","False","False","False","False","False","False","False","False","False","False"]
ok_list = [ ]

#@markdown Choose seasons to download:
S01=False #@param {type:"boolean"}
S02=False #@param {type:"boolean"}
S03=False #@param {type:"boolean"}
S04=False #@param {type:"boolean"}
S05=False #@param {type:"boolean"}
S06=False #@param {type:"boolean"}
S07=False #@param {type:"boolean"}
S08=False #@param {type:"boolean"}
S09=False #@param {type:"boolean"}
S10=False #@param {type:"boolean"}
S11=False #@param {type:"boolean"}
S12=False #@param {type:"boolean"}
check_list[0]=S01
check_list[1]=S02
check_list[2]=S03
check_list[3]=S04
check_list[4]=S05
check_list[5]=S06
check_list[6]=S07
check_list[7]=S08
check_list[8]=S09
check_list[9]=S10
check_list[10]=S11
check_list[11]=S12

for index, s in enumerate(s_list[:]):
  if check_list[index]:
    ok_list.append(s)


for h in downloads:
  if(h.has_metadata()):
    torinfo = h.get_torrent_info()
    files = torinfo.files()
    for x in range(files.num_files()):
      if any(i in files.file_name(x) for i in ok_list):
        h.file_priority(x,4)
      else:
        h.file_priority(x,0)


###Choose ranges

In [ ]:
#@markdown Choose files ranges

#@markdown 

Start=0 #@param {type:"integer"}
End=0 #@param {type:"integer"}

priorities = handle.file_priorities()

for index in range(0,len(priorities)):
  if index >= Start and index <= End:
    handle.file_priority(index,4)
  else:
    handle.file_priority(index,0)

#@markdown 

SecondRange=False #@param {type:"boolean"}
Start=0 #@param {type:"integer"}
End=0 #@param {type:"integer"}

if SecondRange:
  for index in range(0,len(priorities)):
    if index >= Start and index <= End:
      handle.file_priority(index,4)

#@markdown 

ThirdRange=False #@param {type:"boolean"}
Start=0 #@param {type:"integer"}
End=0 #@param {type:"integer"}

if ThirdRange:
  for index in range(0,len(priorities)):
    if index >= Start and index <= End:
      handle.file_priority(index,4)


###Check choosen files

In [ ]:
#@markdown 

#@markdown 

priorities = handle.file_priorities()
torinfo = handle.get_torrent_info()
files = torinfo.files()

print("Files to download:")

for x in range(files.num_files()):
  if (priorities[x]>0):
    print("Download",x,files.file_name(x))

###Force Recheck

In [ ]:
#@markdown 

#@markdown 

handle.force_recheck()

###Add trackers

In [ ]:
#@markdown 

#@markdown 

new_trackers = ["udp://tracker.opentrackr.org:1337/announce"
,"udp://tracker.internetwarriors.net:1337/announce"
,"udp://exodus.desync.com:6969/announce"
,"udp://tracker.cyberia.is:6969/announce"
,"udp://3rt.tace.ru:60889/announce"
,"http://5rt.tace.ru:60889/announce"
,"udp://explodie.org:6969/announce"
,"udp://47.ip-51-68-199.eu:6969/announce"
,"udp://opentracker.i2p.rocks:6969/announce"
,"http://open.acgnxtracker.com:80/announce"
,"udp://www.torrent.eu.org:451/announce"
,"udp://tracker.torrent.eu.org:451/announce"
,"udp://tracker.tiny-vps.com:6969/announce"
,"udp://tracker.ds.is:6969/announce"
,"udp://retracker.lanta-net.ru:2710/announce"
,"udp://open.stealth.si:80/announce"
,"udp://tracker.moeking.me:6969/announce"
,"udp://ipv4.tracker.harry.lu:80/announce"
,"http://tracker.dler.org:6969/announce"
,"http://rt.tace.ru:80/announce"
,"udp://93.158.213.92:1337/announce"
,"udp://138.255.103.83:1337/announce"
,"udp://208.83.20.20:6969/announce"
,"udp://193.218.118.220:6969/announce"
,"udp://79.111.118.59:60889/announce"
,"http://93.157.234.32:60889/announce"
,"udp://184.105.151.164:6969/announce"
,"udp://51.68.199.47:6969/announce"
,"udp://51.81.46.170:6969/announce"
,"udp://91.216.110.52:451/announce"
,"udp://89.234.156.205:451/announce"
,"udp://5.206.60.196:6969/announce"
,"udp://5.226.148.20:6969/announce"
,"udp://37.235.174.46:2710/announce"
,"udp://185.181.60.67:80/announce"
,"udp://138.201.150.56:6969/announce"
,"udp://51.15.40.114:80/announce"
,"http://168.119.237.9:6969/announce"
,"http://195.201.31.194:80/announce"
,"https://trakx.herokuapp.com:443/announce"
,"https://w.wwwww.wtf:443/announce"
,"https://tracker.tamersunion.org:443/announce"
,"https://tracker.sloppyta.co:443/announce"
,"https://tracker.renfei.net:443/announce"
,"https://tracker.nitrix.me:443/announce"
,"https://tracker.nanoha.org:443/announce"
,"https://tracker.lelux.fi:443/announce"
,"https://tracker.imgoingto.icu:443/announce"
,"https://tracker.hama3.net:443/announce"
,"https://tracker.foreverpirates.co:443/announce"
,"http://tracker2.wasabii.com.tw:6969/announce"
,"udp://tracker.sktorrent.net:6969/announce"
,"http://www.wareztorrent.com:80/announce"
,"udp://bt.xxx-tracker.com:2710/announce"
,"udp://tracker.eddie4.nl:6969/announce"
,"udp://tracker.grepler.com:6969/announce"
,"udp://tracker.mg64.net:2710/announce"
,"udp://wambo.club:1337/announce"
,"udp://tracker.dutchtracking.com:6969/announce"
,"udp://tc.animereactor.ru:8082/announce"
,"udp://tracker.justseed.it:1337/announce"
,"udp://tracker.leechers-paradise.org:6969/announce"
,"https://open.kickasstracker.com:443/announce"
,"udp://tracker.coppersurfer.tk:6969/announce"
,"http://87.253.152.137/announce"
,"http://91.216.110.47/announce"
,"http://91.217.91.21:3218/announce"
,"http://91.218.230.81:6969/announce"
,"http://93.92.64.5/announce"
,"http://atrack.pow7.com/announce"
,"http://bt.henbt.com:2710/announce"
,"http://bt.pusacg.org:8080/announce"
,"https://tracker.bt-hash.com:443/announce"
,"udp://tracker.leechers-paradise.org:6969"
,"https://182.176.139.129:6969/announce"
,"udp://zephir.monocul.us:6969/announce"
,"https://tracker.dutchtracking.com:80/announce"
,"https://grifon.info:80/announce"
,"udp://tracker.kicks-ass.net:80/announce"
,"udp://p4p.arenabg.com:1337/announce"
,"udp://tracker.aletorrenty.pl:2710/announce"
,"https://tracker.parrotsec.org:443/announce"
,"https://tracker.moxing.party:6969/announce"
,"https://tracker.ipv6tracker.ru:80/announce"
,"https://tracker.fastdownload.xyz:443/announce"
,"https://gwp2-v19.rinet.ru:80/announce"
,"https://tr.kxmp.cf:80/announce"
,"https://explodie.org:6969/announce"]



###Add tracker single

In [ ]:
#@markdown 

#@markdown 

trackers = handle.trackers()

next_tier = trackers[len(trackers)-1]['tier']

for x in range(len(new_trackers)):
  annouce_entry = {'url': new_trackers[x], 'tier': next_tier}
  handle.add_tracker(annouce_entry) 

###Add trackers multi

In [ ]:
#@markdown 

#@markdown 

for h in downloads:
  trackers = h.trackers()

  next_tier = trackers[len(trackers)-1]['tier']

  for x in range(len(new_trackers)):
    annouce_entry = {'url': new_trackers[x], 'tier': next_tier}
    h.add_tracker(annouce_entry) 


###List trackers

In [ ]:
#@markdown 

#@markdown 

trackers = handle.trackers()
print("Trackers count ",len(trackers))

trackers = handle.trackers()
print("Trackers count ",len(trackers))

for x in trackers:
  #print(x)
  print(x['url'], x['tier'])

###Start Torrent

In [ ]:
#@markdown 

#@markdown 

import time
from IPython.display import display
import ipywidgets as widgets


state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

ses.resume()

for x in downloads:
  x.resume()
  
session_state = ""
if ses.is_paused() :
  session_state = "SESSION IS PAUSED"
else:
  session_state = "SESSION IS RUNNING"

print("            " + session_state)

layout = {'width':'auto', 'margin-left':'50px'}
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_finished():
            s = download.status()
            rate = ""
            rateMB = round(s.download_rate / 1000000, 2)
            if rateMB >= 1:
              rate= "{:.2f} MB".format(rateMB)
            else:
              rate = '{:3d} KB'.format (round(s.download_rate / 1000))

            bar.description = " ".join(
                [
            download.name()+"         ",
                    str(),
                    rate,
                    state_str[s.state],
                ]
            )
            bar.value = round(s.progress * 100)
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            download_bars.remove(bar)
            bar.value = 100
            print(download.name(), "complete")
    time.sleep(1)

###Check status

In [ ]:
#@markdown 

#@markdown 

from IPython.display import display
import ipywidgets as widgets

session_state = ""
if ses.is_paused() :
  session_state = "SESSION IS PAUSED"
else:
  session_state = "SESSION IS RUNNING"

print("            " + session_state)

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = {'width':'auto', 'margin-left':'50px'}
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

for index, download in enumerate(downloads[:]):
    bar = download_bars[index]
    s = download.status()
    state = ""
    if s.paused == True:
      state = "PAUSED"
    else:
      state = "RUNNING"
    bar.description = " ".join(
        [
            download.name(),
            str(round(s.download_rate / 1000000, 2)),
            "MB/s",
            state,
            state_str[s.state]

        ]
    )
    bar.value = round(s.progress * 100)




###Pause session

In [ ]:
#@markdown 

#@markdown 

for h in downloads:
  h.pause()
  
ses.pause()

###Clear downloads

In [ ]:
#@markdown 

#@markdown 

for download in downloads:
  ses.remove_torrent(download, 0)

downloads.clear()